In [27]:
import pandas as pd

In [28]:
overallDataset = pd.read_csv("data/MovieGenre.csv", sep=",", encoding = "ISO-8859-1")
overallDataset = overallDataset[["imdbId", "Genre"]]
original_size = len(overallDataset)

In [29]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(overallDataset, test_size=0.1)
train_size = len(train)
test_size = len(test)

print("The original number of records was " + str(original_size) + ".")
print("The train size was " + str(train_size) + " and the test size was " + str(test_size))
assert original_size == (train_size + test_size)

The original number of records was 40108.
The train size was 36097 and the test size was 4011


In [30]:
test.to_csv("test_labels.csv", index=False)
train.to_csv("train_labels.csv", index=False)